## Notebook for controlling the hexapod

In [ ]:
import numpy as np
from numpy import pi as π, cos, sin, exp, abs, sum
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from meas import *
from numpy.linalg import norm

np.set_printoptions(precision=4, suppress=True)

In [ ]:
# geometric functions
#rototranslation
def rt(p:np.ndarray, r=np.array([0.0, 0.0, 0.0]), t=np.array([0.0,0.0,0.0])): # rotation and translation
    assert p.shape[-1] == 3, f'p shape: {p.shape}'
    assert r.shape == (3,), f'r shape: {r.shape}'
    assert t.shape == (3,), f't shape: {t.shape}'
    p_shape = p.shape
    p = p.reshape(-1, 3)
    rx, ry, rz = r
    Rx = np.array([[1, 0, 0], [0, cos(rx), -sin(rx)], [0, sin(rx), cos(rx)]])
    Ry = np.array([[cos(ry), 0, sin(ry)], [0, 1, 0], [-sin(ry), 0, cos(ry)]])
    Rz = np.array([[cos(rz), -sin(rz), 0], [sin(rz), cos(rz), 0], [0, 0, 1]])    
    # r_xyz = Rz @ Ry @ Rx # rotation matrix
    # p = (r_xyz @ p.T).T + t
    p = (Rx @ Ry @ Rz @ p.T).T + t

    return p.reshape(p_shape)

# projection onto horizontal plane along the normal of the oriented circle
def project_circle_onto_plane(center, radius, rotation_angles, plane_z=0):
    num_points = 100
    angles = np.linspace(0, 2 * np.pi, num_points)

    # 1. Create a circle in the XY plane centered at the origin
    circle_2d = np.array([radius * np.cos(angles), radius * np.sin(angles), np.zeros(num_points)]).T

    # 2. Rotate the circle according to the given rotation angles
    rotated_circle = rt(circle_2d, r=rotation_angles)

    # 3. Translate the rotated circle to the specified center
    circle_points_3d = rotated_circle + center

    # 4. Determine the normal vector of the oriented circle
    # The normal of a circle in the XY plane is [0, 0, 1].
    # We rotate this normal by the same rotation angles.
    initial_normal = np.array([0, 0, 1])
    circle_normal = rt(initial_normal.reshape(1, 3), r=rotation_angles).flatten()

    # Normalize the normal vector
    circle_normal = circle_normal / np.linalg.norm(circle_normal)

    # 5. Project each point of the 3D circle onto the horizontal plane (plane_z)
    projected_points = np.zeros_like(circle_points_3d)

    for i, p_3d in enumerate(circle_points_3d):
        # The line passing through p_3d in the direction of the normal vector
        # is p_3d + t * circle_normal.
        # We want to find t such that the Z-component of this line is plane_z.
        # p_3d.z + t * circle_normal.z = plane_z
        # t * circle_normal.z = plane_z - p_3d.z
        
        # Handle the case where the normal is almost perfectly horizontal (normal_z is close to 0)
        # If normal_z is 0, the circle is vertical and the projection is a line segment,
        # or it's parallel to the plane, in which case it just takes the plane_z.
        if abs(circle_normal[2]) < 1e-6: # If normal is parallel to the plane (or very close)
            projected_points[i] = [p_3d[0], p_3d[1], plane_z]
            # If the circle is exactly vertical, it projects to a line segment.
            # However, the problem states projection *along* the normal, which implies a
            # finite intersection unless normal_z is zero.
            # In this case, we simply project to the z-plane.
        else:
            t = (plane_z - p_3d[2]) / circle_normal[2]
            projected_points[i] = p_3d + t * circle_normal

    return projected_points, circle_points_3d, circle_normal

## Dynamics
HEXA_TOP_REST = np.array([A1T, A2T, C1T, C2T, B1T, B2T])
HEXA_BOT_REST = np.array([A1B, A2B, C1B, C2B, B1B, B2B])

# rototranslation to distances
def rt2d(r=np.array([0.0, 0.0, 0.0]), t=np.array([0.0,0.0,0.0])):
    hexa_top = rt(HEXA_TOP_REST, r, t)
    d = norm(hexa_top - HEXA_BOT_REST, axis=1)
    return d

# distances to rototranslation
def d2rt(d=np.array([D0, D0, D0, D0, D0, D0])):
    r, t = np.array([0.0, 0.0, 0.0]), np.array([0.0,0.0,0.0])
    lr = 1.0e-1 # learning rate
    sr = 1.0e-4 # step for numerical gradient (rotations)
    st = 1.0e-3 # step for numerical gradient (translations)

    def loss(d, d1):
        # return sum((d - d1)**2)
        return sum(abs((d - d1)))

    def grad(r, t):
        l = loss(d, rt2d(r, t))
        x,y,z = t.copy()
        rx,ry,rz = r.copy() 

        gx = (loss(d, rt2d(r, np.array([x+st, y, z]))) - l) 
        gy = (loss(d, rt2d(r, np.array([x, y+st, z]))) - l) 
        gz = (loss(d, rt2d(r, np.array([x, y, z+st]))) - l) 
        grx = (loss(d, rt2d(np.array([rx+sr, ry, rz]), t)) - l)
        gry = (loss(d, rt2d(np.array([rx, ry+sr, rz]), t)) - l)
        grz = (loss(d, rt2d(np.array([rx, ry, rz+sr]), t)) - l)

        # print(f'gx: {gx:.2f}, gy: {gy:.2f}, gz: {gz:.2f}, grx: {grx:.2f}, gry: {gry:.2f}, grz: {grz:.2f}')
        return np.array([grx, gry, grz]), np.array([gx, gy, gz])

    for i in range(5000):
        # print(f'\niter {i}: r: {r}, t: {t}')
        l = loss(d, rt2d(r, t))
        if l < 1e-1: break
        r_grad, t_grad = grad(r, t)
        δr, δt = lr * r_grad, lr * t_grad
        # np.set_printoptions(precision=2, suppress=True)
        # print(f'r_grad: {r_grad}, t_grad: {t_grad} | δr: {δr}, δt: {δt}')
        r -= δr
        t -= δt
        lr *= 0.99 # decay learning rate

    # assert loss(d, rt2d(r, t)) < 1e-1, f'not converged: {loss(d, rt2d(r, t))}'
    if loss(d, rt2d(r, t)) >= 1e-1:
        print(f'not converged: {loss(d, rt2d(r, t))} after {i} iterations')
    # print(f'final loss: {loss(d, rt2d(r, t)):.1f} after {i} iterations')
    return r, t

# test rt2d and d2rt

np.set_printoptions(precision=0)

# test rt2d
# print(f'rt2d at rest: {rt2d()} [mm] should be {D0} [mm]')
print(f'rt2d 0.25 rx: {rt2d(r=np.array([0.25, 0.0, 0.0]))} [mm]')
# print(f'rt2d 0.25 rx/ry: {rt2d(r=np.array([0.25, 0.25, 0.0]))} [mm]')
# print(f'rt2d 0.25 rx + 20 x: {rt2d(r=np.array([0.25, 0.0, 0.0]), t=np.array([20.0, 0.0, 0.0]))} [mm]')

# test d2rt
# print(f'\n\nd2rt at rest: {d2rt()} should be r=0, t=0')
np.set_printoptions(precision=4)
print(f'd2rt rx=0.25   : {d2rt(rt2d(r=np.array([0.25, 0.0, 0.0])))}')
print(f'd2rt rx=ry=0.25: {d2rt(rt2d(r=np.array([0.25, 0.25, 0.0])))}')
# print(f'd2rt 0.25 rx + 20 x: {d2rt(rt2d(r=np.array([0.25, 0.0, 0.0]), t=np.array([20.0, 0.0, 0.0])))} should be r=0.25, t=20 x')    
print(f'd2rt tx=20     : {d2rt(rt2d(r=np.array([0.0, 0.0, 0.0]), t=np.array([20.0, 0.0, 0.0])))}')    


In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# plotting functions
%matplotlib widget

def plot_origin(ax):
    ax.plot([0], [0], [0], 'ro')
    ax.text(0, 0, 0, 'O', color='r')
    m = 500
    # arrows for axes
    L = m/5
    alr = 0.3
    lw = 3.5
    ax.quiver(0, 0, 0, L, 0, 0, color='r', arrow_length_ratio=alr, linewidth=lw)
    ax.quiver(0, 0, 0, 0, L, 0, color='g', arrow_length_ratio=alr, linewidth=lw)
    ax.quiver(0, 0, 0, 0, 0, L, color='b', arrow_length_ratio=alr, linewidth=lw)
    ax.text(L, 0, 0, 'X', color='r')
    ax.text(0, L, 0, 'Y', color='g')
    ax.text(0, 0, L, 'Z', color='b')

    #set axis limits
    ax.set_xlim([-m, m])
    ax.set_ylim([-m, m])
    ax.set_zlim([-m, m])

    return ax

def plot_pipe(ax, r=np.array([0.0, 0.0, 0.0]), t=np.array([0.0, 0.0, 0.0]), d=D1):
    n = 30
    alpha_intersect = 0.3
    θs = np.linspace(0, 2*π, n)
    zs = np.linspace(0, -800, n)
    x,y,z,R,rx,ry,rz = d
    p = np.zeros((n,n,3))
    p[:,:,0] = R * np.cos(θs)[:,None]
    p[:,:,1] = R * np.sin(θs)[:,None]
    p[:,:,2] = zs[None,:]

    # rototranslation fixed wrt top of hexapod
    p = rt(p, r=np.array([rx, ry, rz]), t=np.array([x, y, z]))

    # hexapod rototranslation
    p = rt(p, r=r, t=t)

    for iz in range(n):
        ax.plot(p[:,iz,0], p[:,iz,1], p[:,iz,2], 'b-', alpha=0.3)

    ax.plot(p[:,0,0], p[:,0,1], p[:,0,2], 'b-', alpha=0.3)
    ax.add_collection3d(Poly3DCollection([p[:,0,:]], color='b', alpha=alpha_intersect))
    #project the first disk onnto the D2 and D3 planes
    z2, z3 = D2[2], D3[2]
    center = np.mean(p[:,0,:], axis=0)
    p2,_,_ = project_circle_onto_plane(center=center, radius=R, rotation_angles=np.array([rx, ry, rz])+r, plane_z=z2)
    p3,_,_ = project_circle_onto_plane(center=center, radius=R, rotation_angles=np.array([rx, ry, rz])+r, plane_z=z3)
    ax.plot(p2[:,0], p2[:,1], p2[:,2], 'b--', alpha=alpha_intersect)
    ax.plot(p3[:,0], p3[:,1], p3[:,2], 'b--', alpha=alpha_intersect)
    ax.add_collection3d(Poly3DCollection([p2], color='b', alpha=alpha_intersect))  
    ax.add_collection3d(Poly3DCollection([p3], color='b', alpha=alpha_intersect))
    #project onto the base plane
    p0,_,_ = project_circle_onto_plane(center=center, radius=R, rotation_angles=np.array([rx, ry, rz])+r, plane_z=0)
    ax.plot(p0[:,0], p0[:,1], p0[:,2], '--', color='orange', alpha=alpha_intersect)
    ax.add_collection3d(Poly3DCollection([p0], color='orange', alpha=alpha_intersect))

    return ax

def plot_hexa(ax, r=np.array([0.0, 0.0, 0.0]), t=np.array([0.0, 0.0, 0.0])):
    hexa = rt(HEXA_TOP_REST, r, t)
    a1t, a2t, c1t, c2t, b1t, b2t = hexa
    col = 'orange'
    marker_style = dict(color='orange', linewidth=4, marker='o', markersize=10, markerfacecolor=col, markeredgecolor='k')
    text_style = dict(color='k', fontsize=12, weight='bold', bbox=dict(facecolor=col, edgecolor='k', boxstyle='round,pad=0.2'))
    # A
    ax.plot([a1t[0], A1B[0]], [a1t[1], A1B[1]], [a1t[2], A1B[2]], **marker_style)
    ax.plot([a2t[0], A2B[0]], [a2t[1], A2B[1]], [a2t[2], A2B[2]], **marker_style)
    ax.text(A1B[0], A1B[1], -10, 'A1', **text_style)
    ax.text(A2B[0], A2B[1], -10, 'A2', **text_style)
    # B
    ax.plot([b1t[0], B1B[0]], [b1t[1], B1B[1]], [b1t[2], B1B[2]], **marker_style)
    ax.plot([b2t[0], B2B[0]], [b2t[1], B2B[1]], [b2t[2], B2B[2]], **marker_style)
    ax.text(B1B[0], B1B[1], -10, 'B1', **text_style)
    ax.text(B2B[0], B2B[1], -10, 'B2', **text_style)
    # C
    ax.plot([c1t[0], C1B[0]], [c1t[1], C1B[1]], [c1t[2], C1B[2]], **marker_style)
    ax.plot([c2t[0], C2B[0]], [c2t[1], C2B[1]], [c2t[2], C2B[2]], **marker_style)
    ax.text(C1B[0], C1B[1], -10, 'C1', **text_style)
    ax.text(C2B[0], C2B[1], -10, 'C2', **text_style)
    #base
    hexa_base = np.array([A1B, A2B, C1B, C2B, B1B, B2B])
    hexa_base[:,2] = 0.0
    ax.add_collection3d(Poly3DCollection([hexa_base], color='orange', alpha=0.1))
    return ax

def plot_constraint_disks(ax):
    # D2
    n = 50
    α1 = 0.1
    θs = np.linspace(0, 2*π, n)
    x,y,z,R,rx,ry,rz = D2
    p = np.array([R * np.cos(θs), R * np.sin(θs), np.zeros(n)]).T
    p = rt(p, r=np.array([rx, ry, rz]), t=np.array([x, y, z]))
    ax.plot(p[:,0], p[:,1], p[:,2], 'r-')
    ax.add_collection3d(Poly3DCollection([p], color='r', alpha=α1))  
    # D3
    x,y,z,R,rx,ry,rz = D3
    p = np.array([R * np.cos(θs), R * np.sin(θs), np.zeros(n)]).T
    p = rt(p, r=np.array([rx, ry, rz]), t=np.array([x, y, z]))
    ax.plot(p[:,0], p[:,1], p[:,2], 'r-')
    ax.add_collection3d(Poly3DCollection([p], color='r', alpha=α1))
    return ax


# test
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')
ax.set_box_aspect([1,1,1])

plot_origin(ax)
plot_pipe(ax)
plot_hexa(ax) # at rest
plot_constraint_disks(ax)

# set view angle
ax.view_init(elev=30, azim=210)

plt.show()

# test
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')
# ax.set_aspect('equal')
ax.set_box_aspect([1,1,1])

plot_origin(ax)
random_r = np.array([15 * π/180, 0.0, 0.0])
random_t = np.array([0,0,0])
plot_pipe(ax, r=random_r, t=random_t)
plot_hexa(ax, r=random_r, t=random_t)
plot_constraint_disks(ax)

# set view angle
ax.view_init(elev=30, azim=210)

plt.show()
